# From VEP variants → transcripts → proteins → WT peptide contexts → peptides for NetMHCpan / NetMHCIIpan

This notebook:

- Reads a VEP tab-delimited output file produced with a command like:

  ```bash
  ./vep --af --appris --biotype --buffer_size 500 --check_existing --distance 5000        --mane --polyphen b --pubmed --regulatory --show_ref_allele --sift b        --species homo_sapiens --symbol --transcript_version --tsl --uploaded_allele        --cache --input_file input_data --output_file output_file
  ```

- Keeps **protein-altering** variants (missense, stop gained, frameshift, inframe indels, etc.)
- Picks **one transcript per (variant, gene)** using MANE, APPRIS, TSL, and IMPACT
- Uses **Ensembl REST** to map transcript IDs (ENST) to protein sequences (ENSP)
- Extracts **WT and MUT peptide contexts** around the mutated residue:
    - ±15 aa for MHC-II (31-mers)
    - ±8 aa for MHC-I (17-mers)
- Generates **overlapping WT/MUT peptides**:
    - Class I: 8–11-mers
    - Class II: 13–15-mers
  and keeps only peptides that contain the mutated residue.
- Adds IDs for downstream pairing:
    - `peptide_id`: unique row ID
    - `pair_id`: key that matches WT/MUT windows (same variant, class, length, mut_offset)
    - `mut_offset`: position of the mutated residue within the peptide (0-based)
- Writes:
    - `vep_peptide_contexts.tsv` – variant → transcript → protein → context
    - `peptides_for_netmhc.tsv` – all WT/MUT peptides and metadata
    - `netmhcpan_classI_peptides.txt` – list of class I peptides (for NetMHCpan)
    - `netmhciipan_classII_peptides.txt` – list of class II peptides (for NetMHCIIpan)


In [ ]:
# If needed, install dependencies (uncomment & run once)
# !pip install pandas requests tqdm

import os
import re
import json
import time
from collections import defaultdict

import pandas as pd
import requests
from tqdm.auto import tqdm

# -----------------------
# User configuration
# -----------------------

# Path to your VEP tab-separated output file
# Change this to the path of the VEP file you already have
VEP_FILE = "C:\Users\Kacper\Desktop\immuno\data\qZWuZht1PnMvtm20.txt"  # <-- EDIT THIS

# Output file with peptide contexts
OUTPUT_TSV = "vep_peptide_contexts.tsv"

# Ensembl REST server:
# - For GRCh38: https://rest.ensembl.org
# - For GRCh37: https://grch37.rest.ensembl.org
ENSEMBL_REST_SERVER = "https://rest.ensembl.org"  # change if you used GRCh37

# How many amino acids to keep on each side of the mutated residue
FLANK_MHCII = 15  # → 31-mers
FLANK_MHCI = 8    # → 17-mers


In [ ]:
def read_vep_table(path: str) -> pd.DataFrame:
    """
    Read a VEP tab-delimited output file.

    Assumes the header line starts with '#Uploaded_variation' and skips
    earlier comment lines starting with '##'.
    """
    header_line = None
    header_idx = None
    with open(path, "r") as f:
        for i, line in enumerate(f):
            if line.startswith("#Uploaded_variation"):
                header_line = line.strip().lstrip("#")
                header_idx = i
                break

    if header_line is None:
        raise ValueError("Could not find header line starting with '#Uploaded_variation' in VEP file")

    colnames = header_line.split("\t")
    df = pd.read_csv(
        path,
        sep="\t",
        header=None,
        names=colnames,
        skiprows=header_idx + 1,
        dtype=str  # keep everything as string
    )
    return df


vep_df = read_vep_table(VEP_FILE)
print("Loaded VEP file with shape:", vep_df.shape)
print("First few columns:", list(vep_df.columns)[:15])


In [ ]:
# -----------------------------
# Restrict to protein-altering variants
# -----------------------------

# These are the consequence terms we consider as protein-altering.
# You can adjust this list if needed.
protein_altering_terms = [
    "missense_variant",
    "stop_gained",
    "stop_lost",
    "start_lost",
    "frameshift_variant",
    "inframe_insertion",
    "inframe_deletion",
    "protein_altering_variant",
]


def is_protein_altering(cons: str) -> bool:
    if pd.isna(cons):
        return False
    s = str(cons)
    return any(term in s for term in protein_altering_terms)


# Keep only protein-coding transcripts with protein-altering consequences
if "BIOTYPE" not in vep_df.columns:
    raise ValueError("VEP file is missing BIOTYPE column. Make sure you used --biotype in VEP.")

pc_mask = vep_df["BIOTYPE"] == "protein_coding"
altering_mask = vep_df["Consequence"].apply(is_protein_altering)

# Require that Protein_position and Amino_acids are filled
pos_mask = vep_df["Protein_position"].notna() & (vep_df["Protein_position"] != "-")
aa_mask = vep_df["Amino_acids"].notna() & (vep_df["Amino_acids"] != "-")

vep_pc = vep_df[pc_mask & altering_mask & pos_mask & aa_mask].copy()
print("Protein-coding, protein-altering rows with AA info:", vep_pc.shape)


def parse_aa_change(aa_str: str):
    if pd.isna(aa_str):
        return (None, None)
    s = str(aa_str)
    if "/" in s:
        wt, alt = s.split("/", 1)
        wt = wt or None
        alt = alt or None
        return wt, alt
    return (None, None)


def parse_protein_pos(pos_str: str):
    if pd.isna(pos_str):
        return None
    s = str(pos_str)
    if "-" in s:
        s = s.split("-", 1)[0]  # if it's a range, take the first position
    s = s.strip()
    if not s:
        return None
    try:
        return int(s)
    except ValueError:
        return None


vep_pc["aa_wt"], vep_pc["aa_alt"] = zip(*vep_pc["Amino_acids"].map(parse_aa_change))
vep_pc["prot_pos"] = vep_pc["Protein_position"].map(parse_protein_pos)

# Keep only rows with a clear protein position and WT amino acid
vep_pc = vep_pc[vep_pc["prot_pos"].notna() & vep_pc["aa_wt"].notna()].copy()
vep_pc["prot_pos"] = vep_pc["prot_pos"].astype(int)

print("After parsing AA and positions:", vep_pc.shape)

if vep_pc.empty:
    print("⚠️ No protein-altering variants with AA info found in this file.")


In [ ]:
# --------------------------------------
# Choose one transcript per (variant, gene)
# --------------------------------------

def _is_defined(x) -> bool:
    return pd.notna(x) and str(x) not in {"", "-"}


def _parse_tsl(x):
    if not _is_defined(x):
        return 99
    s = str(x)
    # VEP TSL values can look like '1', 'TSL1', etc.
    s = s.replace("TSL", "")
    try:
        return int(s)
    except ValueError:
        return 99


def _appris_rank(x):
    # Lower is better
    mapping = {
        "P1": 0,
        "P2": 1,
        "P3": 2,
        "P4": 3,
        "P5": 4,
        "P6": 5,
        "A1": 6,  # alternate isoform
    }
    if not _is_defined(x):
        return 99
    return mapping.get(str(x), 99)


def _impact_rank(x):
    mapping = {
        "HIGH": 0,
        "MODERATE": 1,
        "LOW": 2,
        "MODIFIER": 3,
    }
    if not _is_defined(x):
        return 99
    return mapping.get(str(x), 99)


def choose_transcript(group: pd.DataFrame) -> pd.Series:
    """
    Given all VEP rows for one (variant, gene), pick one transcript.

    Priority:
    - MANE_PLUS_CLINICAL (if present)
    - MANE_SELECT
    - Higher IMPACT (HIGH > MODERATE > LOW > MODIFIER)
    - Better APPRIS (P1 > P2 > ...)
    - Better TSL (1 > 2 > ...)
    """
    g = group.copy()

    if "MANE_PLUS_CLINICAL" in g.columns:
        g["_has_mane_plus"] = g["MANE_PLUS_CLINICAL"].map(_is_defined)
    else:
        g["_has_mane_plus"] = False

    if "MANE_SELECT" in g.columns:
        g["_has_mane_select"] = g["MANE_SELECT"].map(_is_defined)
    else:
        g["_has_mane_select"] = False

    g["_impact_rank"] = g["IMPACT"].map(_impact_rank) if "IMPACT" in g.columns else 99
    g["_appris_rank"] = g["APPRIS"].map(_appris_rank) if "APPRIS" in g.columns else 99
    g["_tsl_rank"] = g["TSL"].map(_parse_tsl) if "TSL" in g.columns else 99

    g = g.sort_values(
        by=["_has_mane_plus", "_has_mane_select", "_impact_rank", "_appris_rank", "_tsl_rank"],
        ascending=[False, False, True, True, True],
        kind="mergesort",  # stable sort
    )

    chosen = g.iloc[0].copy()
    for tmp in ["_has_mane_plus", "_has_mane_select", "_impact_rank", "_appris_rank", "_tsl_rank"]:
        if tmp in chosen.index:
            chosen = chosen.drop(tmp)
    return chosen


picked_rows = []
if vep_pc.empty:
    print("No protein-altering rows; picked_df will be empty.")
    picked_df = vep_pc.copy()
else:
    group_cols = ["Uploaded_variation", "SYMBOL"]
    for _, group in tqdm(vep_pc.groupby(group_cols), desc="Choosing transcripts"):
        picked_rows.append(choose_transcript(group))
    picked_df = pd.DataFrame(picked_rows).reset_index(drop=True)

print("After picking one transcript per (variant, gene):", picked_df.shape)


In [ ]:
# --------------------------------------
# Map transcripts (ENST) to translations (ENSP) via Ensembl REST
# --------------------------------------

if "Feature" not in picked_df.columns:
    raise ValueError("VEP file is missing 'Feature' column (transcript ID).")

picked_df["transcript_id"] = picked_df["Feature"].astype(str)

unique_transcripts = sorted(picked_df["transcript_id"].dropna().unique())
print("Unique transcripts to map:", len(unique_transcripts))


def fetch_transcript_to_translation(transcript_ids, server: str = ENSEMBL_REST_SERVER, sleep_between: float = 0.1):
    """
    For each transcript ID (ENST...), call Ensembl REST /lookup/id to find the translation ID (ENSP...).

    Returns
    -------
    dict: {transcript_id -> translation_id}
    """
    if not transcript_ids:
        return {}

    headers = {"Content-Type": "application/json", "Accept": "application/json"}
    mapping = {}

    for tid in tqdm(transcript_ids, desc="Mapping transcripts to translations"):
        if not tid or tid in mapping:
            continue

        # Try with version first, then without version if needed
        candidates = [tid]
        if "." in tid:
            candidates.append(tid.split(".", 1)[0])

        translation_id = None
        for cid in candidates:
            url = server.rstrip("/") + f"/lookup/id/{cid}?expand=1"
            r = requests.get(url, headers=headers)
            if not r.ok:
                continue
            data = r.json()
            trans_info = data.get("Translation") or data.get("translation")
            if trans_info and "id" in trans_info:
                translation_id = trans_info["id"]
                break

        if translation_id is not None:
            mapping[tid] = translation_id

        time.sleep(sleep_between)

    return mapping


transcript_to_translation = fetch_transcript_to_translation(unique_transcripts, server=ENSEMBL_REST_SERVER)
print("Mapped transcripts with translations:", len(transcript_to_translation))

picked_df["translation_id"] = picked_df["transcript_id"].map(transcript_to_translation)
print("Rows without translation:", picked_df["translation_id"].isna().sum())

# Drop rows without translation ID
picked_df = picked_df[picked_df["translation_id"].notna()].copy()
print("Rows after dropping those without translation:", picked_df.shape)


In [ ]:
# --------------------------------------
# Fetch protein sequences for translation IDs (ENSP) via Ensembl REST
# --------------------------------------

unique_proteins = sorted(picked_df["translation_id"].dropna().unique())
print("Unique protein IDs to fetch:", len(unique_proteins))


def fetch_protein_seqs_ensembl(
    protein_ids,
    server: str = ENSEMBL_REST_SERVER,
    chunk_size: int = 50,
    sleep_between: float = 0.1,
) -> dict:
    """
    Fetch protein sequences from Ensembl REST /sequence/id endpoint.

    Parameters
    ----------
    protein_ids : list of str
        Ensembl protein IDs (ENSP...), without or with version.
    server : str
        REST server base URL.
    chunk_size : int
        How many IDs to request per POST.
    sleep_between : float
        Seconds to sleep between chunks (be nice to the server).

    Returns
    -------
    dict: {protein_id -> amino acid sequence}
    """
    protein_ids = [pid for pid in protein_ids if pid]
    if not protein_ids:
        return {}

    unique_ids = list(dict.fromkeys(protein_ids))  # preserve order + unique

    url = server.rstrip("/") + "/sequence/id"
    headers = {"Content-Type": "application/json", "Accept": "application/json"}

    seqs = {}
    for i in tqdm(range(0, len(unique_ids), chunk_size), desc="Fetching protein sequences"):
        chunk = unique_ids[i : i + chunk_size]
        payload = json.dumps({"ids": chunk})
        r = requests.post(url, headers=headers, data=payload)
        if not r.ok:
            print("Error fetching sequences for IDs (showing first 5):", chunk[:5])
            r.raise_for_status()
        data = r.json()
        # data is a list of dicts with 'id' and 'seq' keys
        for entry in data:
            pid = entry.get("id")
            seq = entry.get("seq")
            if pid and seq:
                seqs[pid] = seq
        time.sleep(sleep_between)

    return seqs


protein_seqs = fetch_protein_seqs_ensembl(unique_proteins, server=ENSEMBL_REST_SERVER)
print("Fetched sequences for", len(protein_seqs), "proteins")

picked_df["protein_seq"] = picked_df["translation_id"].map(protein_seqs)
print("Rows with missing protein sequence:", picked_df["protein_seq"].isna().sum())

# Drop rows without sequences
picked_df = picked_df[picked_df["protein_seq"].notna()].copy()
print("Rows after dropping missing sequences:", picked_df.shape)


In [ ]:
# --------------------------------------
# Extract WT peptide contexts (±15, ±8)
# --------------------------------------

def extract_context(seq: str, pos_1based: int, flank: int) -> str:
    """
    Extract a window of amino acids around pos_1based (1-based index).
    Returns a (clipped) window of length <= 2*flank+1.
    """
    if seq is None:
        return None
    try:
        pos0 = int(pos_1based) - 1
    except (TypeError, ValueError):
        return None
    if pos0 < 0 or pos0 >= len(seq):
        return None
    start = max(0, pos0 - flank)
    end = min(len(seq), pos0 + flank + 1)  # end is exclusive
    return seq[start:end]


picked_df["mhcII_wt_context"] = picked_df.apply(
    lambda r: extract_context(r["protein_seq"], r["prot_pos"], FLANK_MHCII),
    axis=1,
)

picked_df["mhcI_wt_context"] = picked_df.apply(
    lambda r: extract_context(r["protein_seq"], r["prot_pos"], FLANK_MHCI),
    axis=1,
)

print(
    picked_df[
        [
            "Uploaded_variation",
            "SYMBOL",
            "Feature",
            "translation_id",
            "aa_wt",
            "aa_alt",
            "prot_pos",
            "mhcII_wt_context",
            "mhcI_wt_context",
        ]
    ].head()
)


In [ ]:
# --------------------------------------
# Optional: mutant peptide contexts (simple missense)
# --------------------------------------

def extract_mutant_context(row, flank: int):
    seq = row["protein_seq"]
    pos = row["prot_pos"]
    aa_alt = row["aa_alt"]
    aa_wt = row["aa_wt"]

    # Only handle simple single-AA substitutions
    if seq is None or pd.isna(pos) or aa_alt is None or aa_wt is None:
        return None
    if len(str(aa_alt)) != 1 or len(str(aa_wt)) != 1:
        return None

    pos0 = int(pos) - 1
    if pos0 < 0 or pos0 >= len(seq):
        return None

    seq_mut = seq[:pos0] + str(aa_alt) + seq[pos0 + 1 :]
    return extract_context(seq_mut, pos, flank)


picked_df["mhcII_mut_context"] = picked_df.apply(
    lambda r: extract_mutant_context(r, FLANK_MHCII),
    axis=1,
)

picked_df["mhcI_mut_context"] = picked_df.apply(
    lambda r: extract_mutant_context(r, FLANK_MHCI),
    axis=1,
)


In [ ]:
# --------------------------------------
# Save variant → transcript → protein → context table
# --------------------------------------

# Core columns
output_cols = [
    "Uploaded_variation",
    "Location",
    "Allele",
    "Consequence",
    "IMPACT",
    "SYMBOL",
    "Gene",
    "Feature",        # transcript ID
    "transcript_id",
    "translation_id", # Ensembl protein ID
    "aa_wt",
    "aa_alt",
    "prot_pos",
    "mhcII_wt_context",
    "mhcI_wt_context",
    "mhcII_mut_context",
    "mhcI_mut_context",
]

# Also keep MANE / APPRIS / TSL if available
for extra in ["MANE_SELECT", "MANE_PLUS_CLINICAL", "MANE", "APPRIS", "TSL"]:
    if extra in picked_df.columns and extra not in output_cols:
        output_cols.append(extra)

# Ensure all requested columns exist (create empty if necessary)
for col in output_cols:
    if col not in picked_df.columns:
        picked_df[col] = pd.NA

output_df = picked_df[output_cols].copy()
output_df.to_csv(OUTPUT_TSV, sep="\t", index=False)

print(f"Saved context table to: {OUTPUT_TSV}")
output_df.head()


## Generate overlapping peptides for NetMHCpan / NetMHCIIpan

We now:

- Generate overlapping sliding windows from the contexts:
  - Class I (NetMHCpan): 8, 9, 10, 11-mers
  - Class II (NetMHCIIpan): 13, 14, 15-mers
- Keep only peptides that contain the mutated residue
- Add:
  - `mut_offset`: 0-based index of mutated residue within the peptide
  - `pair_id`: matches WT/MUT windows for the same variant/class/length/offset
  - `peptide_id`: unique row ID
- Save:
  - `peptides_for_netmhc.tsv`
  - `netmhcpan_classI_peptides.txt`
  - `netmhciipan_classII_peptides.txt`


In [ ]:
# --------------------------------------
# Helper: index of the mutated residue inside each context
# --------------------------------------

def mutated_index_in_context(row, flank: int, context_col: str):
    """
    Given a row with:
      - protein_seq (full protein)
      - prot_pos (1-based position in protein)
      - context_col (mhcI_*_context or mhcII_*_context)
    and the flank used to build that context,
    return the 0-based index of the mutated residue within that context.

    If information is missing, returns None.
    """
    seq = row.get("protein_seq")
    pos = row.get("prot_pos")
    context = row.get(context_col)

    if seq is None or pd.isna(pos) or context is None or pd.isna(context):
        return None

    pos0 = int(pos) - 1  # 0-based in full protein
    if pos0 < 0 or pos0 >= len(seq):
        return None

    start = max(0, pos0 - flank)
    # context was built as: seq[start:end]
    return pos0 - start  # 0-based index within this context


# --------------------------------------
# Helper: generate all overlapping windows that include the mutation
# --------------------------------------

def windows_with_mutation(context: str, mut_idx: int, length: int):
    """
    Return list of (peptide, mut_offset) where:
      - peptide is a substring of `context` of given `length`
      - mut_offset is the 0-based index of the mutated residue within the peptide.

    Only windows that contain position `mut_idx` are returned.
    """
    if context is None or pd.isna(context):
        return []
    n = len(context)
    if n < length:
        return []

    peptides = []
    for start in range(0, n - length + 1):
        end = start + length
        if start <= mut_idx < end:
            pep = context[start:end]
            if len(pep) == length:
                mut_offset = mut_idx - start
                peptides.append((pep, mut_offset))
    return peptides


# --------------------------------------
# Generate peptides for each row in picked_df
# --------------------------------------

def make_peptide_records_for_row(row):
    """
    From one variant / transcript row, generate all WT and MUT peptides
    for:
      - Class I: lengths 8–11 aa
      - Class II: lengths 13–15 aa

    Only peptides that contain the mutated residue are kept.

    For each peptide we store:
      - mut_offset: 0-based index of the mutated residue within the peptide
      - pair_id:   variant+class+length+mut_offset -> used to pair WT vs MUT
    """
    records = []

    # Class I lengths (NetMHCpan)
    classI_lengths = range(8, 12)   # 8, 9, 10, 11
    # Class II lengths (NetMHCIIpan)
    classII_lengths = range(13, 16) # 13, 14, 15

    # Common metadata
    variant_id = row.get("Uploaded_variation")
    gene = row.get("SYMBOL")
    transcript_id = row.get("Feature")  # ENST
    protein_id = row.get("translation_id")
    prot_pos = row.get("prot_pos")
    aa_wt = row.get("aa_wt")
    aa_alt = row.get("aa_alt")

    # ----- Class I: WT and MUT -----
    ctx_I_wt = row.get("mhcI_wt_context")
    ctx_I_mut = row.get("mhcI_mut_context")
    idx_I_wt = mutated_index_in_context(row, FLANK_MHCI, "mhcI_wt_context")
    idx_I_mut = mutated_index_in_context(row, FLANK_MHCI, "mhcI_mut_context")

    for L in classI_lengths:
        # WT
        if idx_I_wt is not None and isinstance(ctx_I_wt, str):
            for pep, mut_offset in windows_with_mutation(ctx_I_wt, idx_I_wt, L):
                pair_id = f"{variant_id}|I|{L}|{mut_offset}"
                records.append({
                    "variant_id": variant_id,
                    "gene": gene,
                    "transcript_id": transcript_id,
                    "protein_id": protein_id,
                    "class": "I",
                    "type": "WT",
                    "length": L,
                    "peptide": pep,
                    "mut_offset": mut_offset,
                    "pair_id": pair_id,
                    "prot_pos": prot_pos,
                    "aa_wt": aa_wt,
                    "aa_alt": aa_alt,
                })
        # MUT
        if idx_I_mut is not None and isinstance(ctx_I_mut, str):
            for pep, mut_offset in windows_with_mutation(ctx_I_mut, idx_I_mut, L):
                pair_id = f"{variant_id}|I|{L}|{mut_offset}"
                records.append({
                    "variant_id": variant_id,
                    "gene": gene,
                    "transcript_id": transcript_id,
                    "protein_id": protein_id,
                    "class": "I",
                    "type": "MUT",
                    "length": L,
                    "peptide": pep,
                    "mut_offset": mut_offset,
                    "pair_id": pair_id,
                    "prot_pos": prot_pos,
                    "aa_wt": aa_wt,
                    "aa_alt": aa_alt,
                })

    # ----- Class II: WT and MUT -----
    ctx_II_wt = row.get("mhcII_wt_context")
    ctx_II_mut = row.get("mhcII_mut_context")
    idx_II_wt = mutated_index_in_context(row, FLANK_MHCII, "mhcII_wt_context")
    idx_II_mut = mutated_index_in_context(row, FLANK_MHCII, "mhcII_mut_context")

    for L in classII_lengths:
        # WT
        if idx_II_wt is not None and isinstance(ctx_II_wt, str):
            for pep, mut_offset in windows_with_mutation(ctx_II_wt, idx_II_wt, L):
                pair_id = f"{variant_id}|II|{L}|{mut_offset}"
                records.append({
                    "variant_id": variant_id,
                    "gene": gene,
                    "transcript_id": transcript_id,
                    "protein_id": protein_id,
                    "class": "II",
                    "type": "WT",
                    "length": L,
                    "peptide": pep,
                    "mut_offset": mut_offset,
                    "pair_id": pair_id,
                    "prot_pos": prot_pos,
                    "aa_wt": aa_wt,
                    "aa_alt": aa_alt,
                })
        # MUT
        if idx_II_mut is not None and isinstance(ctx_II_mut, str):
            for pep, mut_offset in windows_with_mutation(ctx_II_mut, idx_II_mut, L):
                pair_id = f"{variant_id}|II|{L}|{mut_offset}"
                records.append({
                    "variant_id": variant_id,
                    "gene": gene,
                    "transcript_id": transcript_id,
                    "protein_id": protein_id,
                    "class": "II",
                    "type": "MUT",
                    "length": L,
                    "peptide": pep,
                    "mut_offset": mut_offset,
                    "pair_id": pair_id,
                    "prot_pos": prot_pos,
                    "aa_wt": aa_wt,
                    "aa_alt": aa_alt,
                })

    return records


# Apply to all rows
all_records = []
for _, r in picked_df.iterrows():
    all_records.extend(make_peptide_records_for_row(r))

peptides_df = pd.DataFrame(all_records)
print("Total peptides generated:", peptides_df.shape[0])
peptides_df.head()


In [ ]:
# --------------------------------------
# Deduplicate and prepare NetMHCpan / NetMHCIIpan input files (PEPTIDE format)
# --------------------------------------

# Drop exact duplicate rows (same variant/gene/class/type/length/sequence)
peptides_df = peptides_df.drop_duplicates(
    subset=["variant_id", "gene", "class", "type", "length", "peptide"]
).reset_index(drop=True)

# Add a unique peptide_id for reference
peptides_df["peptide_id"] = peptides_df.index.to_series().map(lambda i: f"pep{i:06d}")

print("Peptides after dropping duplicates:", peptides_df.shape[0])

# Split into Class I and Class II
peptides_I = peptides_df[peptides_df["class"] == "I"].copy()
peptides_II = peptides_df[peptides_df["class"] == "II"].copy()

print("Class I peptides:", peptides_I.shape[0])
print("Class II peptides:", peptides_II.shape[0])


def write_plain_list(df, path):
    """
    Write a simple list of peptides:
        PEPTIDE1
        PEPTIDE2
        ...
    This matches the simplest NetMHCpan / NetMHCIIpan format.
    No empty lines are written.
    """
    seqs = [str(p).strip().upper() for p in df["peptide"]]
    with open(path, "w") as f:
        f.write("\n".join(seqs) + "\n")
    print(f"Wrote {len(seqs)} peptides to {path} (plain list)")


# --------- Output paths ---------
NETMHCPAN_TXT   = "netmhcpan_classI_peptides.txt"
NETMHCIIPAN_TXT = "netmhciipan_classII_peptides.txt"
PEPTIDES_TSV    = "peptides_for_netmhc.tsv"

# Write plain lists (PEPTIDE format)
write_plain_list(peptides_I, NETMHCPAN_TXT)
write_plain_list(peptides_II, NETMHCIIPAN_TXT)

# Metadata table for later merging with NetMHC results
# Includes: peptide_id, pair_id, mut_offset, variant_id, gene, class, type, length, peptide, etc.
peptides_df.to_csv(PEPTIDES_TSV, sep="\t", index=False)
print(f"Saved peptide metadata table to {PEPTIDES_TSV}")
peptides_df.head()
